In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import re

In [2]:
x_train = pd.read_csv('/Users/guimb/Documents/Data Scientest/rakuten/data/X_train.csv', sep=',', index_col='Unnamed: 0')
y_train = pd.read_csv('/Users/guimb/Documents/Data Scientest/rakuten/data/Y_train.csv', sep=',', index_col='Unnamed: 0')
df = x_train.join(y_train)

In [3]:
df.head()

,designation,description,productid,imageid,prdtypecode
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,2280
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,1280
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,2705


In [4]:
df["description"].fillna(df["designation"], inplace=True)

In [5]:
# Fusionner les colonnes "désignation" et "description"
df["description"] = df["designation"] + " " + df["description"]

In [6]:
df.columns

Index(['designation', 'description', 'productid', 'imageid', 'prdtypecode'], dtype='object')

### Traduction

In [38]:
from langdetect import detect

# Fonction pour détecter la langue d'une description
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"


# Appliquer la détection de langue à la colonne "description" et créer une nouvelle colonne "langue"
df["langue"] = df["description"].apply(detect_language)

In [45]:
df.columns

Index(['designation', 'description', 'productid', 'imageid', 'prdtypecode',
       'langue'],
      dtype='object')

In [31]:
# Appliquer la détection de langue à la colonne "description" et créer une nouvelle colonne "langue"
langues = df["langue"].unique()
print(langues)

['de' 'fr' 'en' 'ro' 'nl' 'ca' 'es' 'pt' 'it' 'no' 'tl' 'id' 'af' 'so'
 'et' 'da' 'sv' 'fi' 'sw' 'cy' 'vi' 'sl' 'pl' 'hr' 'hu' 'sk' 'lt' 'tr'
 'lv' 'cs' 'sq']


In [49]:
from tqdm import tqdm
from googletrans import Translator


def translate_to_french(row):
    translator = Translator()
    try:
        for col in ["designation", "description"]:
            if row["langue"] != "fr":
                row[col] = translator.translate(row[col], src="auto", dest="fr").text
        return row
    except:
        return row


# Appliquer la traduction en utilisant tqdm pour afficher la progression
tqdm.pandas(desc="Traduction en cours")

# Apply the translation function to each row
df = df.progress_apply(translate_to_french, axis=1)

# Now df contains the translated text in columns 'designation' and 'description'

AttributeError: module 'httpcore' has no attribute 'SyncHTTPTransport'

In [6]:

# Appliquer la détection de langue à la colonne "description" et créer une nouvelle colonne "langue"
langues = df["langue"].unique()
print(langues)

KeyError: 'langue'

### Nettoyage 

In [3]:
# Convertir le texte en minuscules
df["description"] = df["description"].str.lower()

In [4]:
# Supprimez les doublons de la colonne 'description'
df.drop_duplicates(subset=["description"], inplace=True)

# Réindexez le DataFrame
df.reset_index(drop=True, inplace=True)

In [5]:
balises = df['description'].apply(lambda x: re.findall(r'<.*?>', str(x)))
print(balises.explode().unique())

[nan '<br>' '<span class="vga_style2">' '<b>' '</b>' '<strong>'
 '</strong>' '<ul>' '<li>' '</li>' '</ul>' '<br />' '<p>' '</p>' '<i>'
 '</i>' '<span id="result_box" lang="en" tabindex="-1">'
 '<span class="hps">' '</span>' '<span class="hps atn">' '<span>'
 '<p align=&quot;justify&quot;>' '<h4>' '</h4>'
 '<iframe allowfullscreen="" frameborder="0" height="315" src="https://www.youtube.com/embed/iahzt0bebhw" width="560">'
 '</iframe>' '<div>' '</div>' '<div class="rakuten_rich_content" >' '<h2>'
 '</h2>' '<br/>' '<u>' '</u>' '<html>' '</html>' '<h3>' '</h3>'
 '<img src="https://www.deuba24online.de/media/magnalister/products/500px/a_de_102199g1.jpg" style="border:0;" alt="" title="" />'
 '<img src="https://www.deuba24online.de/media/magnalister/products/500px/de_102199d_1_2.jpg" style="border:0;" alt="" title="" />'
 '<img src="https://www.deuba24online.de/media/magnalister/products/500px/de_102199d_2_2.jpg" style="border:0;" alt="" title="" />'
 '<font size=&quot;3&quot;>' '</font>' '

In [6]:
# Supprimer les balise html
df['description'] = df['description'].apply(lambda x: re.sub(r'<.*?>', '', str(x)))


In [11]:
balises = df['description'].apply(lambda x: re.findall(r'<.*?>', str(x)))
print(balises.explode().unique())

[nan]


In [7]:
# Supprimer "&eacute;" et "nan"
df["description"] = df["description"].apply(
    lambda x: re.sub(r"&eacute;|nan|&#39;|&amp;|nbsp;", "", str(x))
)

In [8]:
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
df['description'] = df['description'].apply(lambda x: emoji_pattern.sub('', x))

In [9]:
#Supprimer les mots en double
df['description'] = df['description'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))


In [10]:
# Supprimer les mots vides
df['description'] = df['description'].apply(lambda x: ' '.join(sorted(set(x.split()), key=x.split().index)))


In [11]:
# Supprimer les mots courts
from nltk.corpus import stopwords

def remove_shortwords(texte):
    words = texte.split()
    filter_words = [word for word in words if len(word) > 2] 
    return ' '.join(filter_words)

df['description'] = df['description'].apply(remove_shortwords)


In [12]:
# Appliquer la lemmatisation
import nltk
from nltk import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()

# Fonction de lemmatisation
def lemmatisation(texte):
    mots = texte.split()
    lemmatized = [lemmatizer.lemmatize(mot) for mot in mots]
    return ' '.join(lemmatized)

df['description'] = df['description'].apply(lemmatisation)


[nltk_data] Downloading package wordnet to /Users/guimb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/guimb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
from nltk.corpus import stopwords

# Initialiser la variable des mots vides
stop_words = set(stopwords.words('french'))
stop_words.update([",", "."])

def remove_stopwords(text):
    words = text.split()
    filter_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filter_words)

df['description'] = df['description'].apply(remove_stopwords)

### Machine learning

In [18]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [15]:
X, y = df.description, df.prdtypecode

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
# Réduction de la taille de l'échantillon
sample_size = 1000


# Utilisation d'un sous-ensemble de l'échantillon
X_train = X_train.sample(sample_size, random_state=42)
y_train = y_train.loc[X_train.index]

X_test = X_test.sample(sample_size, random_state=42)
y_test = y_test.loc[X_test.index]

# Vectorisation des descriptions
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Liste des modèles avec leurs paramètres
models = [
    ("SVC", SVC(C=10, kernel="linear")),
    ("LinearSVC", LinearSVC(dual=False, penalty="l2")),
    ("LogisticRegression", LogisticRegression(class_weight="balanced", max_iter=200)),
    ("SGDClassifier", SGDClassifier(penalty="l2")),
    ("KNeighborsClassifier", KNeighborsClassifier(metric="minkowski", n_neighbors=4)),
    (
        "RandomForestClassifier",
        RandomForestClassifier(criterion="entropy", max_depth=5),
    ),
]

# Entraînement et évaluation des modèles avec barre de progression
for model_name, model in models:
    print(f"Training {model_name}...")


    X_train_dense = X_train_vec.toarray()
    X_test_dense = X_test_vec.toarray()


    # Utilisation de tqdm pour suivre l'avancement en pourcentage
    with tqdm(total=100, desc=f"{model_name}", position=0, leave=True) as pbar:
        model.fit(X_train_dense, y_train)
        pbar.update(
            50
        )  # Mise à jour de la barre de progression à 50% (entraînement à 50%)

        y_pred = model.predict(X_test_dense)
        accuracy = accuracy_score(y_test, y_pred)
        pbar.update(
            50
        )  # Mise à jour de la barre de progression à 100% (évaluation à 50%)

    print(f"{model_name}: Accuracy = {accuracy}\n")

Training SVC...


SVC: 100%|██████████| 100/100 [00:31<00:00,  3.22it/s]


SVC: Accuracy = 0.586

Training LinearSVC...


LinearSVC: 100%|██████████| 100/100 [00:00<00:00, 121.16it/s]


LinearSVC: Accuracy = 0.629

Training LogisticRegression...


...

MLPClassifier: Accuracy = 0.378

Training KNeighborsClassifier...


KNeighborsClassifier: 100%|██████████| 100/100 [00:00<00:00, 182.55it/s]


KNeighborsClassifier: Accuracy = 0.521

Training RandomForestClassifier...


RandomForestClassifier: 100%|██████████| 100/100 [00:00<00:00, 266.55it/s]

RandomForestClassifier: Accuracy = 0.291



In [23]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Prétraitement des données
X_train, X_test, y_train, y_test = train_test_split(
    df["description"], df["prdtypecode"], test_size=0.2, random_state=42
)

# Vectorisation des descriptions
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Liste des modèles avec leurs paramètres
models = [
    ("SVC", SVC(C=10, kernel="linear")),
    ("LinearSVC", LinearSVC(dual=False, penalty="l2")),
    ("LogisticRegression", LogisticRegression(class_weight="balanced", max_iter=200)),
    ("SGDClassifier", SGDClassifier(penalty="l2")),
    ("MLPClassifier", MLPClassifier(hidden_layer_sizes=(10,), max_iter=40)),
    ("KNeighborsClassifier", KNeighborsClassifier(metric="minkowski", n_neighbors=4)),
    (
        "RandomForestClassifier",
        RandomForestClassifier(criterion="entropy", max_depth=5),
    ),
]

# Entraînement et évaluation des modèles avec barre de progression
for model_name, model in models:
    print(f"Training {model_name}...")
    model.fit(X_train_vec, y_train)

    print(f"Evaluating {model_name}...")
    y_pred = model.predict(X_test_vec)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{model_name}: Accuracy = {accuracy}\n")

Training SVC...
Evaluating SVC...
SVC: Accuracy = 0.8073278584965256

Training LinearSVC...
Evaluating LinearSVC...
LinearSVC: Accuracy = 0.8158559696778269

Training LogisticRegression...
Evaluating LogisticRegression...
LogisticRegression: Accuracy = 0.7887976416087598

Training SGDClassifier...
Evaluating SGDClassifier...
SGDClassifier: Accuracy = 0.7989050326384503

Training MLPClassifier...


/Users/guimb/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
  warnings.warn(


Evaluating MLPClassifier...
MLPClassifier: Accuracy = 0.7910086333965045

Training KNeighborsClassifier...
Evaluating KNeighborsClassifier...
KNeighborsClassifier: Accuracy = 0.3432301537165719

Training RandomForestClassifier...
Evaluating RandomForestClassifier...
RandomForestClassifier: Accuracy = 0.24236681406611918



## Random Forest

In [19]:
vectorizer = CountVectorizer()
X_train_RF = vectorizer.fit_transform(X_train)
X_test_RF = vectorizer.transform(X_test)


In [20]:
from sklearn.ensemble import RandomForestClassifier

clf_RF = RandomForestClassifier()
clf_RF.fit(X_train_RF,y_train)

RandomForestClassifier()

In [21]:
y_pred_RF = clf_RF.predict(X_test_RF)

In [22]:
from sklearn.metrics import classification_report

# Calcul du rapport de classification
classification_rep = classification_report(y_test, y_pred_RF)

# Afficher le rapport de classification
print(classification_rep)

              precision    recall  f1-score   support

          10       0.50      0.12      0.20         8
          40       1.00      0.08      0.15        12
          50       0.88      0.29      0.44        24
          60       0.77      0.77      0.77        13
        1140       0.53      0.77      0.62        13
        1160       1.00      0.67      0.80         6
        1180       0.00      0.00      0.00         5
        1280       0.40      0.47      0.43        77
        1281       0.32      0.24      0.27        29
        1300       0.65      0.86      0.74        69
        1301       1.00      0.23      0.38        13
        1302       0.08      0.28      0.12        53
        1320       1.00      0.15      0.26        47
        1560       0.49      0.48      0.49        97
        1920       0.83      0.78      0.80        63
        1940       1.00      0.25      0.40        16
        2060       0.40      0.62      0.49        80
        2220       1.00    

/Users/guimb/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/guimb/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/guimb/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

### Deep learning 

In [50]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [20]:
# Prétraitement des données
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["description"])
total_words = len(tokenizer.word_index) + 1

input_sequences = tokenizer.texts_to_sequences(df["description"])
padded_sequences = pad_sequences(input_sequences)

# Encodage des étiquettes
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df["prdtypecode"])
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# Construction du modèle
model = Sequential()
model.add(Embedding(total_words, 100, input_length=len(padded_sequences[0])))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(onehot_encoded.shape[1], activation="softmax"))

# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Division des données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences, onehot_encoded, test_size=0.2, random_state=42
)

# Entraînement du modèle
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Évaluation du modèle
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Loss: {loss}, Accuracy: {accuracy}")

# Prédictions sur de nouvelles données
# Remplacez 'nouvelles_sequences' par les séquences de texte de vos nouvelles données
nouvelles_sequences = tokenizer.texts_to_sequences("Peluche Bob l'éponge")
nouvelles_padded_sequences = pad_sequences(
    nouvelles_sequences, maxlen=len(padded_sequences[0])
)

predictions = model.predict(nouvelles_padded_sequences)

/Users/guimb/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10
2088/2088 [==============================] - 997s 477ms/step - loss: 1.5233 - accuracy: 0.5434 - val_loss: 0.9099 - val_accuracy: 0.7275
Epoch 2/10
2088/2088 [==============================] - 1089s 521ms/step - loss: 0.6267 - accuracy: 0.8097 - val_loss: 0.7495 - val_accuracy: 0.7796
Epoch 3/10
2088/2088 [==============================] - 1121s 537ms/step - loss: 0.3067 - accuracy: 0.9077 - val_loss: 0.7634 - val_accuracy: 0.7849
Epoch 4/10
2088/2088 [==============================] - 1149s 550ms/step - loss: 0.1629 - accuracy: 0.9510 - val_loss: 0.8151 - val_accuracy: 0.7931
Epoch 5/10
2088/2088 [==============================] - 1480s 709ms/step - loss: 0.0933 - accuracy: 0.9720 - val_loss: 0.9231 - val_accuracy: 0.7847
Epoch 6/10
2088/2088 [==============================] - 1205s 577ms/step - loss: 0.0602 - accuracy: 0.9818 - val_loss: 0.9908 - val_accuracy: 0.7893
Epoch 7/10
2088/2088 [==============================] - 1257s 602ms/step - loss: 0.0412 - accuracy: 0.9867 

In [ ]:
# Prédictions sur de nouvelles données
# Remplacez 'nouvelles_sequences' par les séquences de texte de vos nouvelles données
nouvelles_sequences = tokenizer.texts_to_sequences("Peluche Bob l'éponge")
nouvelles_padded_sequences = pad_sequences(
    nouvelles_sequences, maxlen=len(padded_sequences[0])
)

predictions = model.predict(nouvelles_padded_sequences)

In [19]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Prétraitement des données
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["description"])
total_words = len(tokenizer.word_index) + 1

input_sequences = tokenizer.texts_to_sequences(df["description"])
padded_sequences = pad_sequences(input_sequences)

# Encodage des étiquettes
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df["prdtypecode"])
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# Division des données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences, onehot_encoded, test_size=0.2, random_state=42
)

# Construction du modèle avec une couche GRU
embedding_dim = 100
gru_units = 50

model = Sequential()
model.add(Embedding(total_words, embedding_dim))
model.add(GRU(gru_units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(onehot_encoded.shape[1], activation="softmax"))

# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Entraînement du modèle
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Évaluation du modèle
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Loss: {loss}, Accuracy: {accuracy}")

ModuleNotFoundError: No module named 'keras'